In [ ]:
# Importing necessary modules
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Loading  the dataset
df = pd.read_csv("C:/Users/91914/Downloads/weatherAUS.xlsx")
df = pd.DataFrame(df)

# Merging 2 features into one array
def removeNull(Class):
    iterate = df[Class]
    y = []
    mean = df[Class].mean(axis=0)
    for i in iterate:
        if(math.isnan(i)):
            y.append(mean)
        else:
            y.append(i)
    y = pd.DataFrame(y)
    df[Class] = y

removeNull('WindGustSpeed')
removeNull('WindSpeed9am')

# Function to check if a value is NaN
def isNaN(string):
    return string != string

# Handling null values for categorical data
def removeNullCategorical(Class):
    iterate = df[Class]
    y = []
    mostfreq = df[Class].value_counts().idxmax()
    for i in iterate:
        if(isNaN(i)):
            y.append(mostfreq)
        else:
            y.append(i)
    y = pd.DataFrame(y)
    df[Class] = y
    return y

# Encoding categorical data
def labelencode(Class):
    removeNullCategorical(Class)
    temp = df[Class]
    temp = temp.to_numpy()
    temp = temp.flatten()
    LE = LabelEncoder()
    LE.fit(temp)
    array = LE.transform(temp)
    array = pd.DataFrame(array)
    df[Class] = array

# Handling null values for 'RainToday' column
removeNullCategorical('RainToday')
labelencode('RainToday')

# Limiting the data size for analysis
y = df['RainToday'].to_numpy()[:600]
X = df[['WindGustSpeed','WindSpeed9am']].to_numpy()[:600]

# Perceptron
per = Perceptron(tol=1e-3, random_state=0)
per.fit(X, y)
per.score(X, y)

# Hyperparameter tuning
parameters = {'alpha': uniform(loc=0, scale=4)}
clf = RandomizedSearchCV(per, param_distributions=parameters, random_state=0)
search = clf.fit(X, y)
print("best K value:", search.best_params_)

# MLP classifier
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(X, y)

# Hyperparameter tuning
clf = RandomizedSearchCV(mlp, param_distributions=parameters, random_state=0)
search = clf.fit(X, y)
print("best K value:", search.best_params_)

# SVM
svmclass = svm.SVC()
svmclass.fit(X, y)

# Hyperparameter tuning
clf = RandomizedSearchCV(svmclass, param_distributions=parameters, random_state=0)
search = clf.fit(X, y)
print("best K value:", search.best_params_)
